In [ ]:
# source: https://github.com/abduulrahmankhalid/Real-Time-Violence-Detection

In [1]:
import os
import shutil
import cv2
import math
import random
import numpy as np
import datetime as dt
import tensorflow
from collections import deque
import matplotlib.pyplot as plt
plt.style.use("seaborn")

%matplotlib inline
 
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense, Dropout, Flatten, Input, TimeDistributed, Bidirectional
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2
from tensorflow.keras.optimizers import SGD, Adam, Lion

/tmp/ipykernel_3283/3818913170.py:11: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use("seaborn")


# Visualize data

In [ ]:
from IPython.display import HTML
from base64 import b64encode

# To Show a Video in Notebook
def Play_Video(filepath):
    html = ''
    video = open(filepath,'rb').read()
    src = 'data:video/mp4;base64,' + b64encode(video).decode()
    html += '<video width=640 muted controls autoplay loop><source src="%s" type="video/mp4"></video>' % src 
    return HTML(html)

In [ ]:
# Classes Directories
NonViolnceVideos_Dir = "../input/real-life-violence-situations-dataset/Real Life Violence Dataset/NonViolence/"
ViolnceVideos_Dir = "../input/real-life-violence-situations-dataset/Real Life Violence Dataset/Violence/"

# Retrieve the list of all the video files present in the Class Directory.
NonViolence_files_names_list = os.listdir(NonViolnceVideos_Dir)
Violence_files_names_list = os.listdir(ViolnceVideos_Dir)

# Randomly select a video file from the Classes Directory.
Random_NonViolence_Video = random.choice(NonViolence_files_names_list)
Random_Violence_Video = random.choice(Violence_files_names_list)

In [ ]:
Play_Video(f"{NonViolnceVideos_Dir}/{Random_NonViolence_Video}")

In [ ]:
Play_Video(f"{ViolnceVideos_Dir}/{Random_Violence_Video}")

# Extract frames

In [ ]:
IMAGE_HEIGHT, IMAGE_WIDTH = 64, 64
SEQUENCE_LENGTH = 16 # number of frames to be fed to model
DATASET_DIR = "/kaggle/input/real-life-violence-situations-dataset/Real Life Violence Dataset"
CLASSES_LIST = ["NonViolence","Violence"]

In [ ]:
def frames_extraction(video_path):
    frames_list = []

    video_reader = cv2.VideoCapture(video_path)
    
    video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))

    skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)
 
    for frame_counter in range(SEQUENCE_LENGTH):
        # set frame position
        video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
  
        success, frame = video_reader.read() 
 
        if not success:
            break
 
        resized_frame = cv2.resize(frame, (IMAGE_HEIGHT, IMAGE_WIDTH))
        
        normalized_frame = resized_frame / 255
        
        frames_list.append(normalized_frame)
 
    video_reader.release()
 
    return frames_list

# Create dataset

In [ ]:
def create_dataset():
    features = []
    labels = []
    video_files_paths = []

    for class_index, class_name in enumerate(CLASSES_LIST):
        print(f'Extracting Data of Class: {class_name}')
        
        files_list = os.listdir(os.path.join(DATASET_DIR, class_name))
        
        for file_name in files_list:
            video_file_path = os.path.join(DATASET_DIR, class_name, file_name)
 
            frames = frames_extraction(video_file_path)
 
            # check if the extracted frames are equal to the SEQUENCE_LENGTH specified.
            # so ignore the vides having frames less than the SEQUENCE_LENGTH.
            if len(frames) == SEQUENCE_LENGTH:
                features.append(frames)
                labels.append(class_index)
                video_files_paths.append(video_file_path)
 
    features = np.asarray(features)
    labels = np.array(labels)  

    return features, labels, video_files_paths

In [ ]:
# create dataset.
features, labels, video_files_paths = create_dataset()

In [ ]:
# save the extracted data
np.save("features.npy", features)
np.save("labels.npy", labels)
np.save("video_files_paths.npy", video_files_paths)

In [2]:
features = np.load("/kaggle/input/video-feature-np/features.npy")
labels = np.load("/kaggle/input/video-feature-np/labels.npy")
video_files_paths = np.load("/kaggle/input/video-feature-np/video_files_paths.npy")

In [3]:
# convert labels into one-hot vectors
encoded_labels = to_categorical(labels)

In [4]:
x_train, x_temp, y_train, y_temp = train_test_split(features, 
                                                    encoded_labels,
                                                    stratify=encoded_labels,
                                                    test_size=0.3,
                                                    shuffle=True, 
                                                    random_state=2)

x_test, x_val, y_test, y_val = train_test_split(x_temp, 
                                                y_temp, 
                                                stratify=y_temp,
                                                test_size=0.5,
                                                shuffle=True, 
                                                random_state=2)
print(x_train.shape)
print(y_train.shape)
print(x_val.shape)
print(y_val.shape)
print(x_test.shape)
print(y_test.shape)

(1400, 16, 64, 64, 3)
(1400, 2)
(300, 16, 64, 64, 3)
(300, 2)
(300, 16, 64, 64, 3)
(300, 2)


# Create model

In [5]:
def create_model():
    mobile_net = MobileNetV2(
                            input_shape=(64, 64, 3),
                            include_top=False,   
                            weights='imagenet'
                        )
    # make last 40 layers trainable
    mobile_net.trainable = True
    for layer in mobile_net.layers[:-40]:
      layer.trainable = False

    lstm_fw = LSTM(units=32)
    lstm_bw = LSTM(units=32, go_backwards = True)  
    
    model = Sequential(
        [
            Input(shape=(16, 64, 64, 3)),
            TimeDistributed(mobile_net),
            Dropout(0.25),             
            TimeDistributed(Flatten()),
            Bidirectional(lstm_fw, backward_layer = lstm_bw),
            Dropout(0.25),
            Dense(256, activation='relu'),
            Dropout(0.25),
            Dense(128, activation='relu'),
            Dropout(0.25),
            Dense(64, activation='relu'),
            Dropout(0.25),
            Dense(32, activation='relu'),
            Dropout(0.25),
            Dense(2, activation ='softmax')
        ]
    )
    
    return model

In [6]:
MoBiLSTM_model = create_model()

/tmp/ipykernel_3283/809978433.py:2: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  mobile_net = MobileNetV2(


# Fit the model

In [7]:
early_stopping = EarlyStopping(monitor="val_accuracy", 
                                patience=10, 
                                restore_best_weights=True)

MoBiLSTM_model.compile(loss="categorical_crossentropy",
                       optimizer=SGD(learning_rate=0.001), 
                       metrics=["accuracy"])
 
# Fitting the model 
MobBiLSTM_model_history = MoBiLSTM_model.fit(x=x_train, 
                                             y=y_train,
                                             epochs=100,
                                             batch_size=8 ,
                                             shuffle=True, 
                                             validation_data=(x_val, y_val), 
                                             callbacks=[early_stopping])

Epoch 1/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 118s 239ms/step - accuracy: 0.4927 - loss: 0.7044 - val_accuracy: 0.4600 - val_loss: 0.6946
Epoch 2/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 26s 146ms/step - accuracy: 0.4926 - loss: 0.7070 - val_accuracy: 0.5000 - val_loss: 0.6904
Epoch 3/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 25s 146ms/step - accuracy: 0.5163 - loss: 0.6968 - val_accuracy: 0.5300 - val_loss: 0.6906
Epoch 4/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 25s 145ms/step - accuracy: 0.4639 - loss: 0.7037 - val_accuracy: 0.5300 - val_loss: 0.6888
Epoch 5/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 25s 145ms/step - accuracy: 0.5470 - loss: 0.6876 - val_accuracy: 0.5700 - val_loss: 0.6861
Epoch 6/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 25s 145ms/step - accuracy: 0.5054 - loss: 0.6914 - val_accuracy: 0.5533 - val_loss: 0.6851
Epoch 7/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 25s 145ms/step - accuracy: 0.4858 - loss: 0.6967 - val_accuracy: 0.5667 - val_loss: 0.6844
Epoch 8/100
175/175 ━━━━━━━━━━━━━━━━━━━━ 25s 145ms/step - accuracy: 0.5483 

In [8]:
acc = MoBiLSTM_model.evaluate(x_test, y_test)[1]
print(f"{acc:.4f}")

10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 142ms/step - accuracy: 0.8989 - loss: 0.3134
0.8933


In [ ]:
MoBiLSTM_model.save("cnn_lstm.keras")